In [1]:
%%capture
%load_ext autoreload
%autoreload 2
%matplotlib inline

# Parser basics

In [2]:
#|default_exp nbx_parser
#|export
from collections import defaultdict


class Parser(object):
    """
    For us a parser is a callable with signature
    ```
        Iterable -> (a, Iterable) or None
    ```
    where `a` is the parser result. 
    Bindable functions have the signature 
    ```
        a -> Parser a
    ```
    """
    def __init__(self, f = lambda line: (None, line)): 
        """Wraps a fn with signature `line -> (a, line)` into a parser."""
        self.func = f
        
        
    def __call__(self, line): 
        return self.func(line)
    

    def bind(self, f): 
        """Monadic bind. Binds functions `a -> Parser a` to the parser."""
        def q(line):
            """The new parser func."""
            try: b, rest = self(line)
            except: return None
            return f(b)(rest)
        
        return Parser(q)
    
    
    def __rshift__(self, f):
        """Shorthand for 'bind'"""
        return self.bind(f)
    
    
    def __or__(self, q):
        """Creates a new parser that tries `q` if `self` fails."""
        def p(line):
            try: 
                a,rest = self(line)
                return (a, rest)
            except:
                return q(line)
            
        return Parser(p)
    
    
    def __eq__(self, val):
        """Creates a new parser that checks if result equals a given value."""
        def check(a, val):
            if isinstance(val, dict):
                for k, v in val.items(): 
                    if a[k] != v: return False
                return True
            else:
                return a == val

        return self.bind(lambda a: result(a) if check(a, val) else fail)
    
    
    def __ne__(self, val):
        """Checks if result does not equal a given value."""
        def check(a, val):
            if isinstance(val, dict):
                equal = True
                for k, v in val.items(): 
                    if a[k] != v: return True
            else:
                return a != val

        return self.bind(lambda a: result(a) if check(a, val) else fail)
    
        
    def __matmul__(self, key):      
        """
        Returns a bindable function `f(a)` that applies 
        the parser `self` and updates the given result dict `a` 
        with `self`'s results.
        """                
        def f(a):
            def p(line):
                try: b, rest = self(line)
                except: return None
                if key is None: return (a, rest)
                a[key] = b
                return (a, rest)
            
            return p
            
        return f
         

# A couple of useful atomic parsers 
def result(a): return Parser(lambda line: (a, line))
fail = Parser(lambda line: None)
item = Parser(lambda line: (line[0], line[1:]) if len(line)>0 else None)
seed = Parser(lambda line: ({}, line))

In [3]:
#|export
def result_or_none(y:"parser return"):
    """Returns None or the parser result (without the rest)."""
    if y is None: return None
    else: return y[0]

In [4]:
#|export
def seq(p, q): 
    return p >> (lambda a: (q >> (lambda b: result([a, b]))))

In [5]:
#|export
def compr(*ps, f = lambda *x: list(x)):
    def q(line):
        A = []
        rest = line
        for p in ps:
            try: a, rest = p(rest)
            except: return None
            A.append(a)
        return f(*A), rest
    return Parser(q)

In [6]:
p = seq(item, item)
line = "abcd"
assert seq  (item, item)      (line) == (['a', 'b'], 'cd')
assert compr(item, item, item)(line) == (['a', 'b', 'c'], 'd')

In [7]:
#|export
def many(q):
    """Apply `q` ZERO or MORE times."""
    def wrapper(line):
        a = []
        def p(line):
            if len(line) == 0: return (a, line)
            try: b, rest = q(line)
            except: return (a, line)
            a.append(b)
            return p(rest)
        
        return p(line)
    return Parser(wrapper)


def many1(q):
    """Apply `q` ONE or MORE times."""
    def p(line):
        try: a, rest_ = q(line)
        except: return None
        b, rest = many(q)(rest_)
        
        return ([a] + b,rest) 

    return Parser(p)

In [8]:
q = item == "a"
assert many(q)("aabb")  == (["a","a"], "bb")
assert many(q)("bb")    == ([], "bb")
assert many1(q)("aabb") == many(q)("aabb")
assert many1(q)("bb")   == None

# Useful Parsers

## Hashtags `#tag`

In [9]:
#|export
import re
tag_rx = re.compile(r"^\s*#([a-zA-Z_]+)\s*(.*)$")

@Parser
def parse_any_tag(line):
    """
    Splits off a tag (eg. `#MyTAg`) 
    at the beginning of the line. 
    """
    m = tag_rx.match(line)
    if m is not None: return m.groups()
    else: return None

In [10]:
parse_any_tag("#hi my name is")

('hi', 'my name is')

In [11]:
assert  parse_any_tag(         "#hi my name is") == ('hi', 'my name is')
assert (parse_any_tag == "hi")("#hi my name is") == ('hi', 'my name is')
assert (parse_any_tag == "hi")("#hey my name is") == None

In [12]:
#|export
def rx_parser(rx):
    def p(line):
        m = rx.match(line)
        if m is not None: 
            d = m.groupdict()
            return (d, d.pop("rest", ""))
        else: return None
        
    return Parser(p)

In [13]:
p = rx_parser(re.compile(r"^\s*(#)(?P<tag>[a-zA-Z_]+)\s*(?P<rest>.*)$"))
p("#hi my name is")

({'tag': 'hi'}, 'my name is')

## Options, flags `--flag`

In [14]:
#|export
import re
flagname_rx = re.compile(r"\s*(?:[\-]+)(?P<name>[a-zA-Z_\-]+)=?(.*)?$")

def parse_flagname(line):
    """
    Splits off the name of a flag iff 
    there is one at the beginning.
    """
    m = flagname_rx.match(line)
    if m is not None: 
        n,r = m.groups()
        if r is None: r = ""
        return (n,r.lstrip())
    else: return None
    
def starts_with_flag(line): return parse_flagname(line) is not None

In [15]:
print(parse_flagname("  --name mirko"))
print(parse_flagname("--name=mirko"))
print(starts_with_flag("--name mirko"))

('name', 'mirko')
('name', 'mirko')
True


In [16]:
#|export
from collections import defaultdict
import shlex 

@Parser
def parse_flags(line):
    """Extracts flag keys and value pairs"""
    d    = dict()
    last = None
    rest = [] # not a flag-key nor a value 
    for s in shlex.split(line):
        try:
            n, v = parse_flagname(s)
            v = v.strip()
            if v == "":
                d[n] = None
                last = n
            else:
                d[n] = v
                last = None
        except: 
            if last is not None: 
                d[last] = s
                last    = None
            else:
                rest.append(s)
         
    return (d, rest)


parse_tagged_line = seq(parse_any_tag, parse_flags)

In [17]:
print(parse_flags('#nbx --some-name=A B C --something_else=D --some-name=E --fname'))
print(parse_tagged_line('#nbx --some-name=A B C --something_else=D --some-name=E --fname'))

({'some-name': 'E', 'something_else': 'D', 'fname': None}, ['#nbx', 'B', 'C'])
(['nbx', {'some-name': 'E', 'something_else': 'D', 'fname': None}], ['B', 'C'])


# Notebook parsing and tokenizing

In [18]:
#|export
from typing import Union, List, Tuple
from nbx.utils import listmap
import json
from nbx.utils import Bunch, load_nb
from pathlib import Path

In [19]:
#|export 
is_nbx = parse_any_tag == "nbx"


def is_nbx_cell(cell):
    """Checks first of cell source for nbx tag."""
    if cell['cell_type'] != 'code': return False
    if not cell['source']: return False
    line0 = cell['source'][0]
    return is_nbx(line0)


def get_nbx_cells(nb):
    return list(filter(is_nbx_cell, nb.cells))


def get_nbx_cells_src(nb):
    return list(map(lambda c: c["source"] ,filter(is_nbx_cell, nb.cells)))

## Tokenize a cell source

In [20]:
#|export 
class Line(object):
    def __init__(self, src=""):
        self.tag = None
        self.src  = src
        
    def __eq__(self, other): return self.tag == other
    def __ne__(self, other): return self.tag != other
    def __str__(self) : return f"``{self.src}''"
    def __repr__(self): return f"{self.__class__.__name__}('{self.src}')"
    
    
class TaggedLine(Line):
    def __init__(self, tag, flags, src = ""): 
        super().__init__(src=src)
        self.tag  = tag
        self.flags = flags
        
    def __str__(self): 
        flags = "".join([f" --{k}=``{v}''" for k,v in self.flags.items()])
        return f"<{self.tag}{flags}/>"
    
    def __repr__(self): return f"TaggedLine('{self.name}', '{self.src}')"
    
    
class EmptyLine(Line): 
    def __init__(self,): 
        super().__init__(src="")
        self.tag = "empty"
        
    def __str__(self) : return u"\u2205" # "empty set" symbol

In [21]:
#|export 
def tokenize_src(src, tags=None):
    """
    Converts each line of the src 
    into their corresponding `Line` object.
    
    If the `tags` argument is set, only its keys are
    considered valid tags when forming tagged lines.
    """
    parsed = []
    for line in src:
        line = line.rstrip()
        try: 
            (tag, flags), _ = parse_tagged_line(line.rstrip())
            if tags is None or tag in tags:
                parsed.append(TaggedLine(tag, flags, line))
            else:
                parsed.append(Line(line))

        except: 
            if line.strip() == "": parsed.append(EmptyLine())
            else:                  parsed.append(Line(line))
            
    return parsed

In [22]:
nb = load_nb("nbx_example/test_nb_python.ipynb")
nbx_cells = get_nbx_cells(nb)
parsed = tokenize_src(nbx_cells[0]["source"])
for t in parsed: print(t)

<nbx --fname=``test_script.py'' --src=``./src''/>
∅
``print("This message is in the notebook and will be added")''
∅
<xarg/>
``task_id    = -1;''
``result_dir = "./";''
∅
<xarg/>
``xtask    = -1;''
``xdir = "./";''


## Simple notebook parser

In [23]:
#|export 
class Block(object):
    def __init__(self, fname=None, nbpath=None, src=[], nbx_meta={}):
        self.fname  = fname
        self.nbpath = nbpath
        self.src    = src
        self.nbx_meta = nbx_meta
        
    def __str__(self):
        s = f"<Block fname = \"{self.fname}\", nbpath= \"{self.nbpath}\"/>\n"
        for l in block.src: s+=f"\n\t{l}"
        s += "\n"
        return s

In [24]:
#|export 
def parse_into_nbx_blocks(nbpath):
    """
    Parses a notebook into "nbx blocks", a new block is started when 
    an nbx line contains a different filename than the last one.
    
    Typically there is only one block.
    """
    nbpath = Path(nbpath)
    nb     = load_nb(nbpath)
    nbx_cells = get_nbx_cells(nb)
    
    # Convert to Line tokens
    lines = []
    for cell in nbx_cells:
        cell_lines = tokenize_src(cell["source"])
        lines.extend(cell_lines)
        
    # Split into nbx blocks, typically just one.
    last_fname = None
    blocks = []
    for line in lines:
        if line == "nbx" and "fname" in line.flags:
                fname = line.flags["fname"]
                if fname != last_fname:
                    blocks.append(Block(fname=fname, nbpath=nbpath, src=[], nbx_meta=line.flags))
                    last_fname = fname
        blocks[-1].src.append(line)
        
    return blocks

In [25]:
nbpath = "nbx_example/test_nb_julia.ipynb"
blocks = parse_into_nbx_blocks(nbpath)
for block in blocks:
    print(block)

<Block fname = "./_temp/_test.jl", nbpath= "nbx_example/test_nb_julia.ipynb"/>

	<nbx --fname=``./_temp/_test.jl''/>
	``include("src/empty_import.jl")''
	``using Plots''
	∅
	``plot(1:10)''
	<nbx/>
	``println("This IS going into the first file...")''

<Block fname = "_temp/_test_module.jl", nbpath= "nbx_example/test_nb_julia.ipynb"/>

	<nbx --fname=``_temp/_test_module.jl'' --jl_module=``TestModule''/>
	``include("src/empty_import.jl")''
	<some_tag_to_be_parsed --a=``1'' --b=``2''/>
	∅
	``function say_hi()''
	``    println("Hi.")''
	``end''
	``export say_hi''
	∅
	``println("This IS ALSO going in, but into ANOTHER file!")''
	<nbx/>
	``function say_bye()''
	``    println("Bye.")''
	``end''
	``export say_bye''

<Block fname = "./_temp/_test.jl", nbpath= "nbx_example/test_nb_julia.ipynb"/>

	<nbx --fname=``./_temp/_test.jl''/>
	∅
	``print("Add this to FIST as well")''



In [36]:
#|export 
def parse_into_nbx_block_dict(nbpath, tags=["nbx"]):
    """
    Parses a notebook into "nbx blocks", a new block is started when 
    an nbx line contains a different filename than the last one.
    
    Typically there is only one block.
    """
    nbpath = Path(nbpath)
    nb     = load_nb(nbpath)
    nbx_cells = get_nbx_cells(nb)
    
    # Convert to Line tokens
    lines = []
    for cell in nbx_cells:
        cell_lines = tokenize_src(cell["source"])
        lines.extend(cell_lines)
        
    # Split into into nbx blocks, 
    # typically just one.
    fname = None
    blocks = {}
    for line in lines:
        if line == "nbx":
            if "fname" in line.flags: fname = line.flags["fname"]
            if fname not in blocks: 
                    blocks[fname] = Block(fname=fname, nbpath=nbpath, src=[], nbx_meta=line.flags)
                    

        blocks[fname].src.append(line)
        
    return blocks

In [40]:
nbpath = "nbx_example/test_nb_julia.ipynb"
blocks = parse_into_nbx_block_dict(nbpath)

print(list(blocks.keys()))
print(list(blocks.keys())[0] == None)
for block in blocks.values():
    print(block)

[None, './_temp/_test.jl', '_temp/_test_module.jl']
True
<Block fname = "None", nbpath= "nbx_example/test_nb_julia.ipynb"/>

	<nbx/>
	∅
	``println("There is no fname ....")''

<Block fname = "./_temp/_test.jl", nbpath= "nbx_example/test_nb_julia.ipynb"/>

	<nbx --fname=``./_temp/_test.jl''/>
	``include("src/empty_import.jl")''
	``using Plots''
	∅
	``plot(1:10)''
	<nbx/>
	``println("This IS going into the first file...")''
	<nbx --fname=``./_temp/_test.jl''/>
	∅
	``print("Add this to FIST as well")''

<Block fname = "_temp/_test_module.jl", nbpath= "nbx_example/test_nb_julia.ipynb"/>

	<nbx --fname=``_temp/_test_module.jl'' --jl_module=``TestModule''/>
	``include("src/empty_import.jl")''
	<some_tag_to_be_parsed --a=``1'' --b=``2''/>
	∅
	``function say_hi()''
	``    println("Hi.")''
	``end''
	``export say_hi''
	∅
	``println("This IS ALSO going in, but into ANOTHER file!")''
	<nbx/>
	``function say_bye()''
	``    println("Bye.")''
	``end''
	``export say_bye''



In [43]:
#|export 
from nbx.utils import nbx_lib
from nbx.templ import *
import os

TEMPLATES = nbx_lib()/"templates"

def create_script(fname, tpl, vdict={}):
    """Create script from template and value dict""" 
    return create_file_from_template(TEMPLATES/tpl, fname, vdict)

In [44]:
TEMPLATES

PosixPath('/Users/mirko/Workspace/nbx/nbx/templates')

In [45]:
#|export
def relpath(source, target):
    """Returns `source`'s path relative to `target`"""
    source = Path(source)
    target = Path(target)
    cwd = Path(os.getcwd())        
    return Path(os.path.relpath(cwd/source, cwd/target))

In [46]:
sp = Path("../a/b/_include_this_source_file.jl")
tp = Path("../../c/_include_from_here/")
relpath(sp, tp)

PosixPath('../../nbx/a/b/_include_this_source_file.jl')

# Julia

In [47]:
#|export
import re
export_rx = re.compile(r"^export\s+(.+?)(?=\s*;)(.*)")

@Parser
def parse_jl_export(line):
    m = export_rx.match(line)
    if m is not None: 
        exports, rest = m.groups()
        exports = exports.split(",")
        exports = [e.strip() for e in exports] 
        return (exports, rest)
    else: return None

In [48]:
line = "export    func, method ;  "
parse_jl_export(line)

(['func', 'method'], ' ;  ')

In [49]:
#|export
import re
include_rx = re.compile(r"^include\([\"\'\s]+([^\)]*)[\"\'\s]+\)(.*)")

@Parser
def parse_jl_include(line):
    m = include_rx.match(line)
    if m is not None: 
        file, rest = m.groups()
        return (str(Path(file)), rest)
    else: return None

In [50]:
lines = [
    'include(\'src/test.jl\')', 
    'include("src/test.jl")'
]
for line in lines: print(parse_jl_include(line))

('src/test.jl', '')
('src/test.jl', '')


In [51]:
#|export
is_jl_include= lambda line: parse_jl_include(line) is not None

In [52]:
#|export
def transform_jl_include(line, target, rel):
    """Replace the include path by relative path from new origin"""
    
    if not is_jl_include(line): return line
    old, _ = parse_jl_include(line)
    file = old
    if not os.path.isabs(file):
        file = relpath(rel.parent/file, rel.parent/target.parent)
        
    new_line = f"include(\"{file}\"); # autogenerated from: \"{old}\""
    return new_line

In [53]:
line = 'include("src/empty_import.jl")'
nbpath = Path("nbx_example/julia_test.ipynb")
target = Path("../_temp/_test.jl") 
print(transform_jl_include(line, target, nbpath))

include("../nbx_example/src/empty_import.jl"); # autogenerated from: "src/empty_import.jl"


In [54]:
#|export
def nbx_block_to_file(block):
    nbpath = Path(block.nbpath)
    fname  = Path(block.fname)
    (nbpath.parent/fname.parent).mkdir(parents=True, exist_ok=True)
    lines = []

    for line in block.src:
        line.src = transform_jl_include(line.src, fname, nbpath)
        lines.append(line)
        
    # get the actual line src strings and 
    # replace nbx lines with empty ones
    lines = listmap(lambda l: l.src if l!="nbx" else "", lines)
    
    create_script(fname=nbpath.parent/fname, tpl="jl_module.tpl", vdict=dict(lines=lines, nbname=nbpath, **block.nbx_meta))
    
    return os.path.normpath(nbpath.parent/fname)


In [57]:
#|export
from nbx.cli import add_argparse


def _create_jl_from_nb(nb_path):
    nbpath = Path(nb_path)
    created = []
    blocks = parse_into_nbx_block_dict(nbpath)
    for fname,block in blocks.items():
        if fname is not None:
            fname = nbx_block_to_file(block)
            created.append(fname)
    
    print("Exported julia-files:")
    for file in created:
        print(">>", f"\"{file}\"")
    
    return created   

@add_argparse
def create_jl_from_nb(nb_path): return _create_jl_from_nb(nb_path)

In [58]:
nbpath = "nbx_example/test_nb_julia.ipynb"
created = _create_jl_from_nb(nbpath)
created

Exported julia-files:
>> "nbx_example/_temp/_test.jl"
>> "nbx_example/_temp/_test_module.jl"


['nbx_example/_temp/_test.jl', 'nbx_example/_temp/_test_module.jl']

In [59]:
for fname in created:
    !subl $fname